<span style="font-size:24pt; color:black">Notebook for creating AR6 style p-box data</span>

<ul style="font-size:16pt; color:black"> 
Make sure to set the <b>expFolder</b> to point to your facts exp output folder. This notebook creates the following folders within the current directory. 
<li></li>
<li> 1_workflow  </li>
<li> 2_workflow_quantiles </li>
<li> 3_pbox </li>
<li> 4_confidence_level_files </li>
</ul>


In [1]:
import shutil
import os
PD=os.getcwd()
import fun_pbox_Gen_v1 as fn
#
ssps=["ssp119","ssp126","ssp245","ssp370","ssp585"]
#
workflow = ["wf_1e","wf_1f","wf_2e","wf_2f","wf_3e", "wf_3f", "wf_4"]
#
expFolder="/projects/kopp/facts-experiments/221217"
#
region = 'global'

<br>
<span style="font-size: 32pt; color: red; font-family: 'Times New Roman'"> 
    Generate:: <br>
    <ol style="font-size: 12pt; color: green; font-family: 'Times New Roman'">
    <li> Workflow folders</li>
    <li> Quantile files</li>
</ol>  
</span>


In [ ]:
def copy_and_convert(source_file, dest_folder, region):
    shutil.copy2(source_file, dest_folder)
    out_file_path = os.path.join(dest_folder, os.path.basename(source_file).split(".nc")[0] + "_quantiles.nc")
    fn.Samples_to_Quantiles(source_file, out_file_path)

path = os.path.join(os.getcwd(), "1_workflow")
shutil.rmtree(path, ignore_errors=True)
os.makedirs(path)

skip = {
    'wf3e': ['ssp119', 'ssp370'],
    'wf3f': ['ssp119', 'ssp370'],
    'wf4':  ['ssp119', 'ssp245', 'ssp370']
}

for workflow, file_patterns in fn.WF_file_patterns.items():
    for ssp in ssps:
        if ssp in skip.get(workflow, []):
            continue
        
        ssp_path = os.path.join(path, workflow, ssp)
        os.makedirs(ssp_path, exist_ok=True)
        
        # Copy Component files & convert to Quantiles.
        for file in file_patterns:
            component_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{file}_{region}sl.nc'
            copy_and_convert(component_file_path, ssp_path, region)
        
        # Copy Common files & convert to Quantiles.
        for common_file_item in fn.common_files:
            common_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{common_file_item}_{region}sl.nc'
            copy_and_convert(common_file_path, ssp_path, region)
        
        # Copy Total files & convert to Quantiles.
        total_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.total.workflow.{workflow}.{region}.nc'
        copy_and_convert(total_file_path, ssp_path, region)
        
        # Copy VLM file & convert to Quantiles.  
        if region == 'local':
            vlm_file_path = f'{expFolder}/coupling.{ssp}/output/coupling.{ssp}.{fn.local_files}.{region}sl.nc'
            copy_and_convert(vlm_file_path, ssp_path, region)


---
## <font color=red> Generate:: 003_pbox  </font>
---

In [ ]:
Pbox = ["pb_1e","pb_1f","pb_2e","pb_2f"]
#
# Create a folder
Pbox_path = fn.FOcr8("3_pbox")
#
for indv_Pbox0,indv_Pbox1 in enumerate(Pbox):
    pb_path=fn.FOcr8_1(Pbox_path, indv_Pbox1) #create wf folder
    for ssp0,ssp1 in enumerate(ssp):
        # ..........................................................................................
        if indv_Pbox1 in ['pb_2e', 'pb_2f'] and ssp1 in['ssp119','ssp245','ssp370']: continue
        # ..........................................................................................
        ssp_path=fn.FOcr8_1(pb_path, ssp1) #create ssp folders within the pb
        #
        if indv_Pbox1 in ['pb_1e','pb_1f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # larmipAIS (common to pb_1e & pb_1f)
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*larmip.AIS*")
            #
            if indv_Pbox1 in ['pb_1e']:
                # AIS Pbox component
                fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1e");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2e")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

                #
            if indv_Pbox1 in ['pb_1f']:
                # AIS Pbox component.
                fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
                fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
                outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                # total
                srcDIR=PD+"/2_workflow_quantiles/"+"wf_2f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
                fname1=fn.fileNAME0(ssp_path,"total.workflow.wf1f");               fname2=fn.fileNAME0(ssp_path,"total.workflow.wf2f")
                infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}'];      outfile=ssp_path+"/total-workflow.nc"
                fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
                #
                fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
                fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")


        
        if indv_Pbox1 in ['pb_2e']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            # emulandice.AIS
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*emulandice.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.glaciers*");
            
            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)   

            # Glaciers Pbox component
            fname1=fn.fileNAME0(ssp_path,"emulandice.glaciers")
            fname2=fn.fileNAME0(ssp_path,"ipccar5.glaciers")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"glaciers-pb"+indv_Pbox1.split('_')[1]+"-glaciers-"+ssp1+"_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  
            
            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2e");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1e"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3e"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)
            #
            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*glaciers*", "glaciers-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

        if indv_Pbox1 in ['pb_2f']:
            # Copy all WF files.
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_"+indv_Pbox1.split('_')[1]+"/"+ssp1
            fn.cp_dir2dir(srcDIR,ssp_path) 
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*ipccar5.icesheets_AIS*");
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3e/"+ssp1; fn.cp(srcDIR,ssp_path,"*deconto21.AIS*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1;  fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_AIS*"); fn.cp(srcDIR,ssp_path,"*bamber19.icesheets_GIS*");


            # GIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"GrIS1f.FittedISMIP.GrIS")
            fname2=fn.fileNAME0(ssp_path,"bamber19.icesheets_GIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_GIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            # AIS Pbox component
            fname1=fn.fileNAME0(ssp_path,"ipccar5.icesheets_AIS")
            fname2=fn.fileNAME0(ssp_path,"larmip.AIS")
            fname3=fn.fileNAME0(ssp_path,"deconto21.AIS")
            fname4=fn.fileNAME0(ssp_path,"bamber19.icesheets_AIS")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}' , f'{ssp_path}/{fname4}']
            outfile=ssp_path+"/"+"icesheets-pb"+indv_Pbox1.split('_')[1]+"-icesheets-"+ssp1+"_AIS_globalsl.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10)  

            # total
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_1f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_3f/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            srcDIR=PD+"/2_workflow_quantiles/"+"wf_4/"+ssp1; fn.cp(srcDIR,ssp_path,"*total.workflow*")
            fname1=fn.fileNAME0(ssp_path,"total.workflow.wf2f");               
            fname2=fn.fileNAME0(ssp_path,"total.workflow.wf1f"); fname3=fn.fileNAME0(ssp_path,"total.workflow.wf3f"); fname4=fn.fileNAME0(ssp_path,"total.workflow.wf4")
            infiles=[f'{ssp_path}/{fname1}' , f'{ssp_path}/{fname2}', f'{ssp_path}/{fname3}', f'{ssp_path}/{fname4}'];      
            outfile=ssp_path+"/total-workflow.nc"
            fn.main(infiles, outfile, pyear_start=2020, pyear_end=2100, pyear_step=10) 

            fn.delete_files_with_pattern(ssp_path, "*AIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*GIS*", "icesheets-pb*")
            fn.delete_files_with_pattern(ssp_path, "*total*", "*-workflow*")

---
## <font color=red> Generate:: 006_CI-level_OP_files  </font>
---

In [ ]:
pboxdir=PD+'/3_pbox'
outdir = fn.FOcr8("4_confidence_level_files")
fn.GenerateConfidenceFiles(pboxdir, outdir)